# MERGE NEW TERM WITH DM 
rules to merge? 
- if exisiting value exists, and no new value keep
- if exisiting value exists and new value adds information, combine
- if no exisiting value preseent, update with new value

process: 
- describe na before changes
- make changes
- commit changes? 

In [49]:
import pandas as pd

## check what attributes are already in the data model

In [112]:
import os

In [113]:
cwd = Path(os.getcwd())

ROOT_DIR_NAME = "ELITE-data-models"

for p in cwd.parents:
    if bool(re.search(ROOT_DIR_NAME + "$", str(p))):
        print(p)
        ROOT_DIR = p

/Users/nlee/Documents/Projects/ELITE/ELITE-data-models


In [50]:
new_term_path = "../../data/rfc_tables_cleaned/genotyping_assay_cleaned_rfc.csv"
new_term_df = pd.read_csv(new_term_path)

print('--- New Term ---')
display(new_term_df.head())

dm = pd.read_csv('../../EL.data.model.csv')

dm['columnType'] = dm['columnType'].str.upper()

print('--- Exisiting Data Model ---')
display(dm.head())

--- New Term ---


,Attribute,Description,Valid Values,Required,Module,columnType,Ontology,Notes,Properties,DependsOn,Parent,UsedIn
0,genotyping_assay,Template for genotyping_assay,NaN,False,Template,NaN,Sage Bionetworks,NaN,NaN,"Component,Filename,OtherMeasurementTechnique ,...",Component,NaN
1,measurementTechnique,The measurement technique describing the assay...,"16S rRNA gene sequencing,1D Gel,2D Gel,Array,B...",False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay
2,OtherMeasurementTechnique,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifyMeasurementTechnique,measurementTechnique,genotyping_assay
3,OtherSampleType,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifySampleType,sampleType,genotyping_assay
4,reagentCatalogNumber,"If the assay reagent is a commercial product, ...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay


--- Exisiting Data Model ---


,Attribute,Description,Valid Values,DependsOn,Properties,Required,Parent,DependsOn Component,Source,Validation Rules,Module,Ontology,UsedIn,columnType,Notes,multivalue
0,10x,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,ValidValues,NaN,NaN,NaN,NaN,NaN
1,10x Visium Spatial Gene Expression,Instrument Model Name,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,Instrument,NaN,NaN,NaN,NaN,NaN
2,Accel-NGS 2S Plus,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,ValidValues,NaN,NaN,NaN,NaN,NaN
3,Accel-NGS Methyl-Seq,NaN,NaN,NaN,NaN,False,ValidValue,NaN,NaN,NaN,ValidValues,NaN,NaN,NaN,NaN,NaN
4,acquisitionBatchID,"Acquisition batch identifier, provided by the ...",NaN,NaN,ManifestColumn,False,ManifestColumn,NaN,Sage Bionetworks,NaN,"Assay,Metadata",Sage Bionetworks,Metabolomics,STRING,NaN,NaN


In [82]:
new_term_df['exists'] = new_term_df['Attribute'].isin(dm['Attribute'])

new_terms_to_add = new_term_df[~new_term_df["exists"]]

new_terms_to_add

,Attribute,Description,Valid Values,Required,Module,columnType,Ontology,Notes,Properties,DependsOn,Parent,UsedIn,exists
0,genotyping_assay,Template for genotyping_assay,NaN,False,Template,NaN,Sage Bionetworks,NaN,NaN,"Component,Filename,OtherMeasurementTechnique ,...",Component,NaN,False
2,OtherMeasurementTechnique,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifyMeasurementTechnique,measurementTechnique,genotyping_assay,False
3,OtherSampleType,"When column = `other`, add your custom value t...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,specifySampleType,sampleType,genotyping_assay,False
6,reagentIDs,"One or more identifiers, separated by a semico...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay,False
19,treatmentIDs,"One or more identifiers, separated by a semico...",NaN,False,Metadata,STRING,"Sage Bionetworks,ImmPort",NaN,ManifestColumn,NaN,NaN,genotyping_assay,False
25,YesUseReagent,"When column = `Yes`, add your custom value to ...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,"reagentCatalogNumber,reagentContact,reagentIDs...",useReagent,genotyping_assay,False
26,YesUseTreatment,"When column = `Yes`, add your custom value to ...",NaN,False,Other,STRING,Sage Bionetworks,NaN,ValidValue,"treatmentAmountUnit,treatmentAmountValue,treat...",useTreatment,genotyping_assay,False


In [52]:
import numpy as np

In [53]:
# cleanup 
dm['columnType'] = dm['columnType'].fillna('').str.split(',').apply(lambda x: ','.join(np.unique(x))).replace('', np.nan)

In [54]:
dm['UsedIn'] = dm['UsedIn'].str.replace('genotyping', 'genotyping_assay')

In [71]:
index_keys = ['Attribute']
keys = ['dm', 'new_term']

df1 = dm[dm["Attribute"].isin(new_term_df["Attribute"])].set_index(index_keys).sort_index()
df2 = (
    new_term_df[new_term_df["Attribute"].isin(dm["Attribute"])]
    .set_index(index_keys)
    .sort_index()
)


df = pd.concat(
    [df1, df2],
    keys=keys,
    axis=1,
).sort_index(level=1, axis=1)

# easier to process
df = df.stack(future_stack=True).dropna(how = 'all', axis = 0)

# if new term does not have value then also drop since it provides no info
df = df[~df['new_term'].isna()]

# if dm is empty, then fill with new term
df['dm'] = df['dm'].fillna(df['new_term'])

# remove duplicates
df = df[df['dm'] != df['new_term']]

df.index = df.index.set_names(["Attribute", "column"])

df

dm  \
Attribute                   column                                                            
measurementTechnique        Description   The name of the measurement technique describi...   
                            Ontology                                       Sage Bionetworks   
                            Required                                                   True   
                            UsedIn                                  Metabolomics,proteomics   
reagentCatalogNumber        Module                                                    Assay   
reagentContact              Module                                                    Assay   
reagentLotNumber            Module                                                    Assay   
reagentManufacturer         Module                                                    Assay   
reagentName                 Module                                                    Assay   
reagentWeblink              Module                                                    Assay   
sampleType                  Description   The type of sample collected or the term used ...   
                            Notes         Cell and ChEBI ontologies.,A data contributor ...   
                            Ontology                                       Sage Bionetworks   
                            UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...   
                            Valid Values  Appendix, B cell, Basophils, Bone, Bone Marrow...   
specifyMeasurementTechnique UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...   
specifySampleType           UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...   
specimenID                  Description   Identifying string linked to a particular samp...   
                            UsedIn        Biospecimen human,Biospecimen nonHuman,Metabol...   
useReagent                  Description                 Was a reagent applied to the sample   
                            Valid Values                                   No,YesUseReagent   
useTreatment                Description               Was a treatment applied to the sample   
                            Valid Values                                 No,YesUseTreatment   

                                                                                   new_term  
Attribute                   column                                                           
measurementTechnique        Description   The measurement technique describing the assay...  
                            Ontology                               Sage Bionetworks,ImmPort  
                            Required                                                  False  
                            UsedIn                                         genotyping_assay  
reagentCatalogNumber        Module                                                 Metadata  
reagentContact              Module                                                 Metadata  
reagentLotNumber            Module                                                 Metadata  
reagentManufacturer         Module                                                 Metadata  
reagentName                 Module                                                 Metadata  
reagentWeblink              Module                                                 Metadata  
sampleType                  Description                        The type of sample collected  
                            Notes         The sample types are adopted from Uberon, Cell...  
                            Ontology                               Sage Bionetworks,ImmPort  
                            UsedIn                                         genotyping_assay  
                            Valid Values  Amniotic Fluid,Appendix,B cell,Basophils,Bone,...  
specifyMeasurementTechnique UsedIn                                         genotyping_assay  
specifySampleType    

In [77]:
def combine_str_cols(df, combo_col): 
    # Expand list cols
    df.loc[df.index.get_level_values("column") == combo_col, "dm"] = df.loc[
        df.index.get_level_values("column") == combo_col, :
    ].agg(",".join, axis=1)

    df.loc[df.index.get_level_values("column") == combo_col, "dm"] = df.loc[
        df.index.get_level_values("column") == combo_col, "dm"
    ].apply(lambda x: ",".join(list(np.unique([s.strip() for s in x.split(",")]))))

    # df = df.drop(combo_col, level=1, axis=0)

    return df

In [78]:
df = combine_str_cols(df, 'Valid Values')
df = combine_str_cols(df, "Ontology")
df = combine_str_cols(df, "UsedIn")

In [79]:
df

dm  \
Attribute                   column                                                            
measurementTechnique        Description   The name of the measurement technique describi...   
                            Ontology                               ImmPort,Sage Bionetworks   
                            Required                                                   True   
                            UsedIn                 Metabolomics,genotyping_assay,proteomics   
reagentCatalogNumber        Module                                                    Assay   
reagentContact              Module                                                    Assay   
reagentLotNumber            Module                                                    Assay   
reagentManufacturer         Module                                                    Assay   
reagentName                 Module                                                    Assay   
reagentWeblink              Module                                                    Assay   
sampleType                  Description   The type of sample collected or the term used ...   
                            Notes         Cell and ChEBI ontologies.,A data contributor ...   
                            Ontology                               ImmPort,Sage Bionetworks   
                            UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...   
                            Valid Values  Amniotic Fluid,Appendix,B cell,Basophils,Bone,...   
specifyMeasurementTechnique UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...   
specifySampleType           UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...   
specimenID                  Description   Identifying string linked to a particular samp...   
                            UsedIn        Biospecimen human,Biospecimen nonHuman,Metabol...   
useReagent                  Description                 Was a reagent applied to the sample   
                            Valid Values                                   No,YesUseReagent   
useTreatment                Description               Was a treatment applied to the sample   
                            Valid Values                                 No,YesUseTreatment   

                                                                                   new_term  
Attribute                   column                                                           
measurementTechnique        Description   The measurement technique describing the assay...  
                            Ontology                               Sage Bionetworks,ImmPort  
                            Required                                                  False  
                            UsedIn                                         genotyping_assay  
reagentCatalogNumber        Module                                                 Metadata  
reagentContact              Module                                                 Metadata  
reagentLotNumber            Module                                                 Metadata  
reagentManufacturer         Module                                                 Metadata  
reagentName                 Module                                                 Metadata  
reagentWeblink              Module                                                 Metadata  
sampleType                  Description                        The type of sample collected  
                            Notes         The sample types are adopted from Uberon, Cell...  
                            Ontology                               Sage Bionetworks,ImmPort  
                            UsedIn                                         genotyping_assay  
                            Valid Values  Amniotic Fluid,Appendix,B cell,Basophils,Bone,...  
specifyMeasurementTechnique UsedIn                                         genotyping_assay  
specifySampleType    

In [80]:
df = df.drop(columns=['new_term'])
df

dm
Attribute                   column                                                         
measurementTechnique        Description   The name of the measurement technique describi...
                            Ontology                               ImmPort,Sage Bionetworks
                            Required                                                   True
                            UsedIn                 Metabolomics,genotyping_assay,proteomics
reagentCatalogNumber        Module                                                    Assay
reagentContact              Module                                                    Assay
reagentLotNumber            Module                                                    Assay
reagentManufacturer         Module                                                    Assay
reagentName                 Module                                                    Assay
reagentWeblink              Module                                                    Assay
sampleType                  Description   The type of sample collected or the term used ...
                            Notes         Cell and ChEBI ontologies.,A data contributor ...
                            Ontology                               ImmPort,Sage Bionetworks
                            UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...
                            Valid Values  Amniotic Fluid,Appendix,B cell,Basophils,Bone,...
specifyMeasurementTechnique UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...
specifySampleType           UsedIn        Metabolomics,Microbiome,RNAseq,Whole Genome Se...
specimenID                  Description   Identifying string linked to a particular samp...
                            UsedIn        Biospecimen human,Biospecimen nonHuman,Metabol...
useReagent                  Description                 Was a reagent applied to the sample
                            Valid Values                                   No,YesUseReagent
useTreatment                Description               Was a treatment applied to the sample
                            Valid Values                                 No,YesUseTreatment

In [91]:
df['dm']

Attribute                    column      
measurementTechnique         Description     The name of the measurement technique describi...
                             Ontology                                 ImmPort,Sage Bionetworks
                             Required                                                     True
                             UsedIn                   Metabolomics,genotyping_assay,proteomics
reagentCatalogNumber         Module                                                      Assay
reagentContact               Module                                                      Assay
reagentLotNumber             Module                                                      Assay
reagentManufacturer          Module                                                      Assay
reagentName                  Module                                                      Assay
reagentWeblink               Module                                                      Assay
sampleTy

In [86]:
dm_stacked = dm.set_index('Attribute').stack(future_stack=True)

In [93]:
dm_stacked.update(df['dm'])

In [106]:
dm_unstacked = dm_stacked.unstack()

In [107]:
dm_unstacked.loc["measurementTechnique"]

Description            The name of the measurement technique describi...
Valid Values                                                         NaN
DependsOn                                                            NaN
Properties                                                ManifestColumn
Required                                                            True
Parent                                                               NaN
DependsOn Component                                                  NaN
Source                                                               NaN
Validation Rules                                                     NaN
Module                                                          Metadata
Ontology                                        ImmPort,Sage Bionetworks
UsedIn                          Metabolomics,genotyping_assay,proteomics
columnType                                                        STRING
Notes                  A data contributor should be

In [104]:
# dm_unstacked = dm_unstacked.reset_index()
# dm_unstacked["Attribute"] = dm_unstacked["Attribute"].apply(lambda x: re.sub('\(|\)|\?', '', x))
# dm_unstacked = dm_unstacked.drop(columns=["index"], errors="ignore")
# dm_unstacked = dm_unstacked.set_index('Attribute')
# dm_unstacked

In [109]:
from pathlib import Path

In [114]:
dm_unstacked.to_csv(Path(ROOT_DIR, 'EL.data.model.csv'))

In [ ]:
# choices: no change, update, merge

In [115]:
import subprocess

In [122]:
proc = subprocess.Popen(
    """ schematic schema convert EL.data.model.csv """, cwd=ROOT_DIR, shell = True
)

proc.communicate()

if proc.returncode == 0: 
    print('Success!')
else: 
    print('FAIL')

Starting schematic...
Parsing data model.
Generating data model graph.
Validating the data model internally.
error: Schematic requires models be a directed acyclic graph (DAG). Please inspect your model.
Converting data model to JSON-LD
By default, the JSON-LD output will be stored alongside the first input CSV or JSON-LD file. In this case, it will appear here: 'EL.data.model.jsonld'. You can use the `--output_jsonld` argument to specify another file path.


The Data Model was created and saved to 'EL.data.model.jsonld' location.
Execution time: 00:50 (M:S)


(None, None)

0